# Create a SQLite DataBase 


In [11]:
import fiona
import sqlite3
import pandas as pd
import os

sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Check if the database exists
if os.path.exists(sqlite_db_path):
    # Delete the existing database
    os.remove(sqlite_db_path)
    print(f"Existing database '{sqlite_db_path}' has been deleted.")

# Define the paths to the files
gpkg_path = 'Resources/California_County_Boundaries.gpkg'
fires_csv_path = 'Outputs/fire_stats_year_county.csv'
extracted_csv_path = 'Outputs/extracted_data.csv'

# Create a SQLite database connection
conn = sqlite3.connect(sqlite_db_path)
cursor = conn.cursor()

# Load the CSV files into SQLite using pandas
fires_df = pd.read_csv(fires_csv_path)
extracted_df = pd.read_csv(extracted_csv_path)

fires_df.to_sql('fires', conn, if_exists='replace', index=False)
extracted_df.to_sql('extracted', conn, if_exists='replace', index=False)

# Use Fiona to read the GeoPackage and extract the attribute data
with fiona.open(gpkg_path, layer=0) as layer:
    # Read the records and their properties (attributes) into a DataFrame
    records = [feature['properties'] for feature in layer]
    gpkg_df = pd.DataFrame(records)

# Load the extracted attribute data into SQLite
gpkg_df.to_sql('california_county_boundaries', conn, if_exists='replace', index=False)

# Perform the JOIN operation with explicit column selection and renaming to avoid duplication
final_query_with_explicit_columns = """
    SELECT ej.County as Extracted_County, ej.*, f.County as Fire_County, f.*
    FROM (
        SELECT e.*, g.*
        FROM extracted e
        JOIN california_county_boundaries g
        ON e.County = g.COUNTY_NAME
    ) ej
    JOIN fires f
    ON ej.County = f.County
"""

# Execute the final query
final_df = pd.read_sql_query(final_query_with_explicit_columns, conn)

# Remove the duplicated columns
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

# Save the final result to a new table
final_df.to_sql('final_merged_data', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

# Provide the path to the SQLite database
print("SQLite database saved to:", sqlite_db_path)


Existing database 'Outputs/DB/fire_data.sqlite' has been deleted.
SQLite database saved to: Outputs/DB/fire_data.sqlite


## Running Test Queries

In [25]:
import sqlite3
import pandas as pd
import os

# Check if the file exists
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'
if os.path.exists(sqlite_db_path):
    # Connect to the SQLite database
    conn = sqlite3.connect(sqlite_db_path)
    
    # Query to check for unique Fire Names in Alameda County
    query = """
    SELECT
        County,
        Year,
        "Fire Name"
    FROM
        fires
    WHERE
        County = 'San Mateo'
    ORDER BY
        Year, "Fire Name";
    """
    
    # Execute the query and load the result into a DataFrame
    alameda_fire_names_df = pd.read_sql_query(query, conn)
    
    # Close the connection
    conn.close()
    
    # Display the unique Fire Names for Alameda County by year
    print(alameda_fire_names_df.head(20))  # Display the first 20 rows to inspect the data
else:
    print("The database file does not exist at the specified path:", sqlite_db_path)


      County  Year                      Fire Name
0  San Mateo  2008                    Martin Fire
1  San Mateo  2008                         Quarry
2  San Mateo  2008                        Trabing
3  San Mateo  2020  CZU AUGUST LIGHTNING\nCOMPLEX


In [27]:
import sqlite3
import pandas as pd

# Path to your SQLite database
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Define the SQL query
query = """
SELECT
    County,
    Year,
    COUNT(DISTINCT "Fire Name") AS Total_Fires
FROM
    fires
GROUP BY
    County, Year
ORDER BY
    County, Year;
"""

# Execute the query and load the result into a DataFrame
fire_counts_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the result
print(fire_counts_df)


      County  Year  Total_Fires
0    Alameda  2010            1
1    Alameda  2011            1
2    Alameda  2015            1
3    Alameda  2018            1
4    Alameda  2022            1
..       ...   ...          ...
452     Yuba  2011            1
453     Yuba  2016            1
454     Yuba  2017            2
455     Yuba  2020            3
456     Yuba  2021            1

[457 rows x 3 columns]


In [14]:
import sqlite3
import pandas as pd

# Path to your SQLite database
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Query to get the column names of the final_merged_data table
query = "PRAGMA table_info(final_merged_data);"

# Execute the query and load the result into a DataFrame
columns_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the columns
print(columns_df)

    cid              name     type  notnull dflt_value  pk
0     0  Extracted_County     TEXT        0       None   0
1     1            County     TEXT        0       None   0
2     2        Tot_Damage     REAL        0       None   0
3     3              Year  INTEGER        0       None   0
4     4     COUNTY_ABBREV     TEXT        0       None   0
5     5       COUNTY_CODE     TEXT        0       None   0
6     6       COUNTY_FIPS     TEXT        0       None   0
7     7       COUNTY_NAME     TEXT        0       None   0
8     8        COUNTY_NUM  INTEGER        0       None   0
9     9          GlobalID     TEXT        0       None   0
10   10            ISLAND     TEXT        0       None   0
11   11       Fire_County     TEXT        0       None   0
12   12         Fire Name     TEXT        0       None   0
13   13             Start     TEXT        0       None   0
14   14         Contained     TEXT        0       None   0
15   15             Acres  INTEGER        0       None  

In [18]:
import sqlite3
import pandas as pd

# Path to your SQLite database
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Query to list all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
tables = conn.execute(query).fetchall()

# Close the connection
conn.close()

# Display the tables
print(tables)

[('fires',), ('extracted',), ('california_county_boundaries',), ('final_merged_data',)]


In [28]:
import sqlite3
import pandas as pd

# Path to your SQLite database
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Define the SQL query to calculate total acres burned
query = """
SELECT
    County,
    Year,
    SUM(Acres) AS Total_Acres_Burned
FROM
    fires
GROUP BY
    County, Year
ORDER BY
    County, Year;
"""

# Execute the query and load the result into a DataFrame
acres_burned_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the result
print(acres_burned_df)


      County  Year  Total_Acres_Burned
0    Alameda  2010                 475
1    Alameda  2011                1045
2    Alameda  2015                2850
3    Alameda  2018                 480
4    Alameda  2022                 552
..       ...   ...                 ...
452     Yuba  2011                 360
453     Yuba  2016                 389
454     Yuba  2017               10939
455     Yuba  2020                2411
456     Yuba  2021                 939

[457 rows x 3 columns]


In [35]:
import pandas as pd
import sqlite3

# Path to your SQLite database
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Load the data from the database into a DataFrame
df = pd.read_sql_query("SELECT County, Year, Start, Contained FROM fires", conn)

# Convert the 'Start' and 'Contained' columns to datetime
df['Start'] = pd.to_datetime(df['Start'])
df['Contained'] = pd.to_datetime(df['Contained'])

# Function to generate a set of active days for a fire
def get_active_days(row):
    return pd.date_range(start=row['Start'], end=row['Contained'])

# Apply the function to each row to generate the active days
df['Active_Days'] = df.apply(get_active_days, axis=1)

# Group by 'Year' and 'County' and union all active days, explicitly excluding the grouping columns
def union_active_days(group):
    all_days = set().union(*group['Active_Days'])
    return pd.Series({'Unique_Fire_Days': len(all_days)})

# Apply the union function with include_groups=False
unique_fire_days = df.groupby(['Year', 'County'], group_keys=False).apply(lambda x: union_active_days(x)).reset_index()

# Store the results in the database
unique_fire_days.to_sql('unique_fire_days', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print("Unique fire days per county per year have been stored in the 'unique_fire_days' table in your database.")


Unique fire days per county per year have been stored in the 'unique_fire_days' table in your database.


/var/folders/_d/q6dl733j0hdfhxbg883rg3xc0000gn/T/ipykernel_8881/1757942905.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_fire_days = df.groupby(['Year', 'County'], group_keys=False).apply(lambda x: union_active_days(x)).reset_index()


In [36]:
import sqlite3
import pandas as pd

# Path to your SQLite database
sqlite_db_path = 'Outputs/DB/fire_data.sqlite'

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_db_path)

# Define the SQL query
query = """
SELECT
    County,
    Year,
    Unique_Fire_Days
FROM
    unique_fire_days
ORDER BY
    County, Year;
"""

# Execute the query and load the result into a DataFrame
unique_fire_days_df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the result
print(unique_fire_days_df)


      County  Year  Unique_Fire_Days
0    Alameda  2010                 2
1    Alameda  2011                 3
2    Alameda  2015                 7
3    Alameda  2018                 2
4    Alameda  2022                 1
..       ...   ...               ...
452     Yuba  2011                 2
453     Yuba  2016                 2
454     Yuba  2017                13
455     Yuba  2020                 4
456     Yuba  2021                 9

[457 rows x 3 columns]


In [ ]:
# Check the columns in the shapefile
print("Columns in counties_gdf:", counties_gdf.columns)

# Check the columns in the database query result
print("Columns in data:", data.columns)
